<a href="https://colab.research.google.com/github/vitroid/PythonTutorials/blob/master/2%20Advanced/320RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 教師あり学習の例


## 決定木による分類

(Pythonデータサイエンスハンドブックp.423)

次のような2次元のデータを考える。データの種類によって4つの色が割りあてられている。

In [ ]:
import matplotlib.pyplot as plt

from sklearn.datasets import make_blobs

X,y = make_blobs(n_samples=300, centers=4, random_state=0, cluster_std=1.0) #散布データを生成する便利な関数

plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='rainbow')

上の例では、Xに各点の座標が、yには色(0〜3)が入っている。

In [ ]:
y

これらの点の集合に対し、境界線を引きたい。人間にやらせれば簡単に目分量で引くだろうが、これを機械にやらせたい。

最も安易な方法として、水平あるいは垂直な線で全体をおおまかに二分する。分割に際しては、色をランダムにひとつ選び、その色と、他の色の峻別が最大になるような位置で分ける。(多数決原理)

(言葉で書くと簡単そうだが、自分で書くとけっこう面倒臭い。水平に境界をひくのが良いか、垂直のほうが良いか、どの色を選ぶべきかなど、うまくやらないと境界線が決まらなかったりする)

指定された回数に達するか、領域に1つの色しか含まれなくなるまでこれを繰りかえす。

ここでは、Scikit-learnに含まれるDecisionTreeClassifier(決定木分類器)を使うことにする。

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier().fit(X, y)

以下の関数を使って、二分した結果を表示する。

In [ ]:
def visualize_classifier(model, X, y, ax=None, cmap='rainbow'):
    ax = ax or plt.gca()
    # 訓練データをプロット
    plt.scatter(X[:, 0], X[:, 1], c=y, s=30, cmap=cmap, clim=(y.min(), y.max()), zorder=3)
    ax.axis('tight')
    ax.axis('off')
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    # 推定値へのあてはめ
    model.fit(X, y)
    xx,yy = np.meshgrid(np.linspace(*xlim, num=200), np.linspace(*ylim, num=200))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

    # 色分け表示
    n_classes = len(np.unique(y))
    contours = ax.contourf(xx, yy, Z, alpha=0.3,
                          levels=np.arange(n_classes+1) - 0.5,
                          cmap=cmap, #clim=(y.min(), y.max()),
                           zorder=1)
    ax.set(xlim=xlim, ylim=ylim)


In [ ]:
visualize_classifier(DecisionTreeClassifier(max_depth=1), X, y)

In [ ]:
visualize_classifier(DecisionTreeClassifier(max_depth=2), X, y)

In [ ]:
visualize_classifier(DecisionTreeClassifier(max_depth=3), X, y)

In [ ]:
visualize_classifier(DecisionTreeClassifier(max_depth=4), X, y)

こんな風に、データをつぎつぎに二分して細分していくデータ構造をツリーと呼ぶ。(この絵では木には見えないが)

これを見ると、「なんと回りくどい手順だ。もっと直観的に境界線を引けそうなのに」と思うかもしれない。しかし、上の例は2次元データだから画面上で境界が明らかに見えるが、例えばあとででてくるような64次元データになると、直感など何の役にも立たないことは容易に想像できる。機械学習の強みは、人間の直感をはるかに超えた多次元データであってもそれなりに境界を自動的に決められる点にある。

上の例では、4段階まで分割を行い、それなりに良い境界線を得た。調子にのってこのまま完全に分類ができるまで細分していくとどうなるか。

In [ ]:
visualize_classifier(DecisionTreeClassifier(), X, y)

境界線が不自然に複雑になってしまうのがわかると思う。これはいわゆる「過学習」である。(以前、多項式フィッティングで、不必要に複雑な曲線が得られたケースを思いだしてほしい)

過学習に至ると、与えられた訓練データに対しては非常によいフィッティング性能を示すが、未知データへのあてはめで失敗する可能性が高くなる(例えば、上の図に見られる赤の小領域など)

決定木分類器はこのように過学習に至りやすい。この弱点を補うために考えられたのが次のランダムフォレスト法である。

### ランダムフォレスト法
決定木分類器において、縦割りや横割りの順序、選択する色をランダムに変化させると、境界線はその都度変化する。それらを集計し、統計的に境界線を決定するのがランダムフォレスト法である。(フォレスト「森」という名前は木をたくさん生成するところから来ていると思われる)
上のデータに対し、ランダムフォレスト法を適用すると、かなりそれらしい境界線が得られる。

In [ ]:
from sklearn.ensemble        import RandomForestClassifier

visualize_classifier(RandomForestClassifier(n_estimators=100), X, y)

この結果では、一見すると決定木分類器の結果とそれほど見た目が変わらないので、ランダムフォレスト法の威力を実感できないかもしれない。そこで、もっと実用的な問題に挑戦する。

### ランダムフォレスト法による文字分類

手書き文字を学習させ、分類器を作る。

手書き文字は8x8ピクセルのグレースケール画像で、それぞれの文字のデータが教師データとして与えられている。

64個の数値の列から、文字を割りだすと考えると、これは64次元での分類問題と言える。

学習用データを表示してみる。

In [ ]:
# taken from Python Data Science handbook

%matplotlib inline

import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
digits = load_digits()

# 画像データの表示
fig, ax = plt.subplots(8, 8, figsize=(6, 6))
fig.subplots_adjust(left=0,right=1, bottom=0,top=1,hspace=0.05,wspace=0.05)
for i, axi in enumerate(ax.flat):
    axi.imshow(digits.images[i], cmap='binary')
    axi.text(0,7,str(digits.target[i]))
    axi.set(xticks=[], yticks=[])

過学習かどうかを判定できるように、データを2つに分ける。

In [ ]:
# 学習用データとテストデータの分割
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(digits.data, digits.target, random_state=0)

学習用データを使って、ランダムフォレスト分類器を訓練する

In [ ]:
from sklearn.ensemble        import RandomForestClassifier

model = RandomForestClassifier(n_estimators=1000)
model.fit(Xtrain, ytrain)

それを使って、未知データの推定を行う。

In [ ]:
ypred = model.predict(Xtest)

# print(digits.images)
# print(Xtest)
# 画像データの表示
fig, ax = plt.subplots(12, 12, figsize=(6, 6))
fig.subplots_adjust(left=0,right=1, bottom=0,top=1,hspace=0.05,wspace=0.05)
for i, axi in enumerate(ax.flat):
    axi.set(xticks=[], yticks=[])

    if ypred[i] != ytest[i]:
        axi.text(0,7,str(ypred[i]), color='red')
        axi.imshow((Xtest[i]).reshape(8,8), cmap='Reds')
    else:
        axi.imshow(Xtest[i].reshape(8,8), cmap='binary')



from sklearn import metrics
print(metrics.classification_report(ypred, ytest))

144個のテストデータで4つを読み違えたので、正答率は97%である。(読み間違えた文字は人間でも分類が難しく、計算機に同情する)

プログラムする人が何の工夫もすることなく、全自動でデータを学習させるだけで、これだけ分類できるのはなかなか大したものと言えるのではないだろうか。



# 応用1

名前から、性別を推定できるかどうかを見てみましょう。

世界中の名前のデータセットが公開されていて、pythonで簡単にアクセスできます。

元データはこちら: https://github.com/philipperemy/name-dataset

In [ ]:
pip install names-dataset

まずは、データセットの中身を覗いてみます。

In [ ]:
from names_dataset import NameDataset, NameWrapper
nd = NameDataset()
# 日本人の名前の上位1000000個をとってくる
data = nd.get_top_names(n=1000000, country_alpha2='JP')
data

漢字とアルファベットの名前が混在しているので、アルファベットに限定します。(日本人なら、名前を聞くだけでだいたい聞きわけられると想定)



In [ ]:
data["JP"].keys()

In [ ]:
import re

def alphabetic_names(names):
    pattern = r'[^a-zA-Z]'
    for name in names:
        alphaname = re.sub(pattern, "", name)
        # アルファベットのみで10文字以内の名前に限定。
        if alphaname != "" and len(alphaname) <= 10:
            yield alphaname.lower()

names_m = [x for x in alphabetic_names(data["JP"]["M"])]
names_f = [x for x in alphabetic_names(data["JP"]["F"])]

names_m

ところで、両方に含まれる名前はあるでしょうか。

In [ ]:
set(names_m) & set(names_f)

日本語は、男性名と女性名が明確に分かれていて、予測しやすいのかもしれません。

これらを使い、教師データを作ります。単純に、女性なら0、男性なら1としましょう。

In [ ]:
import numpy as np

gender_m = [1] * len(names_m)
gender_f = [0] * len(names_f)

名前はアルファベットの列で、しかも長さが不統一です。これを、同じ長さの数値ベクトルにできれば、画像などと同じような扱いができて便利そうです。

とりあえず、文字コードにおきかえ、足りない部分は0で埋めます。


In [ ]:
def encode(name, size=10):
    v = [0] * (size - len(name))
    for c in name:
        v.append(ord(c) - 96)
    return v

encode("masakazu", 10)

In [ ]:
vectors_m = [encode(name) for name in names_m]
vectors_f = [encode(name) for name in names_f]

vectors_m

男性データと女性データを統合します。

In [ ]:
X = np.array(vectors_m + vectors_f)
Y = np.array(gender_m + gender_f)

訓練用データと評価データに分けます。



In [ ]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest =  train_test_split(X, Y, test_size=0.2)
ytest

さあ、これで名前と性別の対応が準備できました。

文字認識で使った、RandomForest分類器を使って、名前と性別の対応を学習させます。

In [ ]:
from sklearn.ensemble        import RandomForestClassifier

model = RandomForestClassifier(n_estimators=1000)
model.fit(Xtrain, ytrain)

これを使って、評価データを推定します。ypredは、Xtestにある名前が女性か男性かを推定した結果です。

In [ ]:
ypred = model.predict(Xtest)
ypred

混同行列を作ります。これは、予測と正解がどれぐらい一致しているかを見るものです。

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(ytest, ypred)

cm

女性名320個のうち、58%の185個は正しく女性名だと判定されましたが、135個は男性名だと誤解した、という意味です。男性の名前は男性と判別しやすかったようです。


## これは何に使える?

* 定型的な作業で、訓練が必要なもの。
* 次元が多すぎて、統計処理もままならないデータの自動ラベル付け。
* 教師データが十分たくさんある場合。

実際には、教師データを準備するのが一番大変である。機械に教えこむために必要なデータには、人間が正解を全部準備してやる必要がある。機械学習の業界では、分類器の性能を評価するための標準的な教師データがいろいろ準備されてはいるが、自分の扱いたいデータ用の教師データは自分で準備する必要がある。